## 영화 5개 입력받기

In [ ]:
print("좋아하는 영화 5개를 입력해주세요(구분은 ,로): ")
movie_list = input().split(",")
print(movie_list)

좋아하는 영화 5개를 입력해주세요(구분은 ,로): 
패신저스,헝거게임: 판엠의 불꽃,어벤져스,해리포터와 불의 잔,트와일라잇
['패신저스', '헝거게임: 판엠의 불꽃', '어벤져스', '해리포터와 불의 잔', '트와일라잇']


## 영화별 출연진 정보 가져오기

영화 목록에서 영화코드 추출

In [ ]:
import requests
import json

In [ ]:
movie_cd_list = []
director_list = []
for movie in movie_list:
  movie_cd_url = 'https://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key=d99e474ec53733a372d8413bd2753506&movieNm='+movie
  res = requests.get(movie_cd_url)
  text = res.text
  d = json.loads(text)
  if d['movieListResult']['totCnt'] > 1:    # 동명의 영화가 있을 경우
    item_director, item_year = input(movie+"의 감독과 개봉연도를 입력하세요(구분은 ,로): ").split(",")   # 감독과 개봉연도로 구분
    for item in d['movieListResult']['movieList']:
      for director in item['directors']:
        if item_director == director['peopleNm'] and item_year == item['openDt'][:4]:
          director_list = director['peopleNm']
          movie_cd_list.append([item['movieCd'],
                                item['movieNm'],
                                director_list,
                                item['openDt'][:4]])
  if d['movieListResult']['totCnt'] == 1:
    for item in d['movieListResult']['movieList']:
      for director in item['directors']:
        director_list = director['peopleNm']
      movie_cd_list.append([item['movieCd'],
                            item['movieNm'],
                            director_list,
                            item['openDt'][:4]])
  
print(movie_cd_list)

패신저스의 감독과 개봉연도를 입력하세요(구분은 ,로): 모튼 틸덤,2017
어벤져스의 감독과 개봉연도를 입력하세요(구분은 ,로): 조스 웨던,2012
트와일라잇의 감독과 개봉연도를 입력하세요(구분은 ,로): 캐서린 하드윅,2008
[['20161084', '패신저스', '모튼 틸덤', '2017'], ['20124127', '헝거게임: 판엠의 불꽃', '게리 로스', '2012'], ['20124721', '어벤져스', '조스 웨던', '2012'], ['20050310', '해리 포터와 불의 잔', '마이크 뉴웰', '2005'], ['20081697', '트와일라잇', '캐서린 하드윅', '2008']]


In [ ]:
import pandas as pd

movie_cdlist_df = pd.DataFrame(movie_cd_list)
movie_cdlist_df.columns = ['movieCd', 'movieNm','directors', 'openYr']
movie_cdlist_df

,movieCd,movieNm,directors,openYr
0,20161084,패신저스,모튼 틸덤,2017
1,20124127,헝거게임: 판엠의 불꽃,게리 로스,2012
2,20124721,어벤져스,조스 웨던,2012
3,20050310,해리 포터와 불의 잔,마이크 뉴웰,2005
4,20081697,트와일라잇,캐서린 하드윅,2008


영화코드로 배우추출

In [ ]:
movie_detail_list = []
movie_director_list = []
movie_cast_list = []    # 한 영화당 최대 10명의 배우를 추출하여 저장한 리스트
direct_list = []
movie_mainactor_list = []    # 한 영화당 최대 2명(주인공)의 배우를 추출하여 저장한 리스트
actor_list = []              # 5개 영화의 movie_cast_list을 모은 것
mainactor_list = []          # 5개 영화의 movie_mainactor_list을 모은 것
movie_actor_list = [[] for i in range(0,5)]
movie_direc_list = [[] for i in range(0,5)]
j=0

for movieCd in movie_cdlist_df['movieCd']:
  movie_director_list.clear()
  movie_cast_list.clear()
  movie_mainactor_list.clear()
  actor_list_url = 'https://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key=d99e474ec53733a372d8413bd2753506&movieCd='+movieCd
  res = requests.get(actor_list_url)
  text = res.text
  d = json.loads(text)

  for direc in d['movieInfoResult']['movieInfo']['directors']:
    movie_director_list.append(direc['peopleNm'])
  direct_list+=movie_director_list

  for cast in d['movieInfoResult']['movieInfo']['actors']:
    if len(movie_mainactor_list) < 2:
      movie_mainactor_list.append(cast['peopleNm'])
    if len(movie_cast_list) <= 9:
      movie_cast_list.append(cast['peopleNm'])
  #print(movie_cast_list)
  actor_list += movie_cast_list
  mainactor_list += movie_mainactor_list

  j+=1

print(direct_list)
print(actor_list)
print(mainactor_list)

['모튼 틸덤', '게리 로스', '조스 웨던', '마이크 뉴웰', '캐서린 하드윅']
['크리스 프랫', '제니퍼 로렌스', '마이클 쉰', '앤디 가르시아', '로렌스 피쉬번', '제니퍼 로렌스', '조쉬 허처슨', '리암 헴스워스', '엘리자베스 뱅크스', '로버트 다우니 주니어', '스칼렛 요한슨', '크리스 헴스워스', '크리스 에반스', '제레미 레너', '다니엘 래드클리프', '루퍼트 그린트', '엠마 왓슨', '랄프 파인즈', '로버트 패틴슨', '크리스틴 스튜어트', '로버트 패틴슨']
['크리스 프랫', '제니퍼 로렌스', '제니퍼 로렌스', '조쉬 허처슨', '로버트 다우니 주니어', '스칼렛 요한슨', '다니엘 래드클리프', '루퍼트 그린트', '크리스틴 스튜어트', '로버트 패틴슨']


배우 등장 횟수 세기

In [ ]:
actor_dict = dict()
for i in actor_list:
  if i not in actor_dict:
    actor_dict[i]=1
  else:
    actor_dict[i] = actor_dict[i]+1

actor_dict

{'다니엘 래드클리프': 1,
 '랄프 파인즈': 1,
 '로렌스 피쉬번': 1,
 '로버트 다우니 주니어': 1,
 '로버트 패틴슨': 2,
 '루퍼트 그린트': 1,
 '리암 헴스워스': 1,
 '마이클 쉰': 1,
 '스칼렛 요한슨': 1,
 '앤디 가르시아': 1,
 '엘리자베스 뱅크스': 1,
 '엠마 왓슨': 1,
 '제니퍼 로렌스': 2,
 '제레미 레너': 1,
 '조쉬 허처슨': 1,
 '크리스 에반스': 1,
 '크리스 프랫': 1,
 '크리스 헴스워스': 1,
 '크리스틴 스튜어트': 1}

주연배우(출연진리스트 맨 앞 2사람) 등장 횟수

In [ ]:
mainactor_dict = dict()
for i in mainactor_list:
  if i not in mainactor_dict:
    mainactor_dict[i]=1
  else:
    mainactor_dict[i] = mainactor_dict[i]+1

mainactor_dict

{'다니엘 래드클리프': 1,
 '로버트 다우니 주니어': 1,
 '로버트 패틴슨': 1,
 '루퍼트 그린트': 1,
 '스칼렛 요한슨': 1,
 '제니퍼 로렌스': 2,
 '조쉬 허처슨': 1,
 '크리스 프랫': 1,
 '크리스틴 스튜어트': 1}

추천해줄 배우 5명 선별
* 2번 이상 등장한 배우 우선 선발
* 빈자리는 랜덤으로 고름

In [ ]:
import random

prefer_actor_list = []
for key, value in actor_dict.items():
  if value > 1:
    prefer_actor_list.append(key)
if len(prefer_actor_list)<5:
  for i in range(5-len(prefer_actor_list)):
    actor = random.choice(mainactor_list)
    if actor not in prefer_actor_list:
      prefer_actor_list.append(actor)
  
prefer_actor_list

['제니퍼 로렌스', '로버트 패틴슨', '조쉬 허처슨', '스칼렛 요한슨', '로버트 다우니 주니어']

배우 코드 추출

In [ ]:
actor_code = []
filmo_list = []

for prefer_actor in prefer_actor_list:
  actor_list_url = 'http://kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleList.json?key=d99e474ec53733a372d8413bd2753506'+'&peopleNm='+prefer_actor
  res = requests.get(actor_list_url)
  text = res.text
  d = json.loads(text)

  for actor in d['peopleListResult']['peopleList']:
    if actor['repRoleNm'] == '배우':
      filmo_list = actor['filmoNames'].split("|")
      actor_code.append([actor['peopleCd'], actor['peopleNm'],actor['peopleNmEn'],
                         actor['repRoleNm'], filmo_list[:5]])

In [ ]:
actor_code_df = pd.DataFrame(actor_code)
actor_code_df.columns = ['peopleCd','peopleNm','peopleNmEn','repRoleNm','filmoNames']
actor_code_df

,peopleCd,peopleNm,peopleNmEn,repRoleNm,filmoNames
0,10062627,제니퍼 로렌스,Jennifer Lawrence,배우,"[엑스맨: 다크 피닉스, 레드 스패로, 마더!, 패신저스, 엑스맨: 아포칼립스]"
1,20111975,로버트 패틴슨,Robert Pattinson,배우,"[테넷, 하이 라이프, 굿타임, 잃어버린 도시 Z, 퀸 오브 데저트]"
2,10066309,조쉬 허처슨,Josh Hutcherson,배우,"[엘리엇과 산타 썰매단, 헝거게임: 더 파이널, 헝거게임: 모킹제이, 파라다이스 로..."
3,10038163,스칼렛 요한슨,Scarlett Johansson,배우,"[블랙 위도우, 씽2게더, 조조 래빗, 결혼 이야기, 어벤져스: 엔드게임]"
4,10016538,로버트 다우니 주니어,Robert Downey Jr.,배우,"[닥터 두리틀, 어벤져스: 엔드게임, 어벤져스: 인피니티 워, 스파이더맨: 홈 커밍..."
